<a href="https://colab.research.google.com/github/futartup/S8-assignment/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import all the custom and inbuilt modules

In [2]:
try:
  import ipynb
except:
  !pip install ipynb --upgrade
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import os, sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/S8')
from torchsummary import summary
import torch.optim as optim
import torch.nn as nn
import torchvision

# custom apis
from ipynb.fs.full.transform_train_test_loader import *
from ipynb.fs.full.conf import *
from ipynb.fs.full.test import *
from ipynb.fs.full.train import *

# The model related code. Load the model from the conf file. 

In [3]:

if custom_model['use']:
  #use custom model
  pass
elif inbuilt_model['use']:
  from ipynb.fs.full.model.pytorch_resnet import *
  from ipynb.fs.full.model.resnet import *
  model_type = inbuilt_model['model']
  #net = globals()[model_type]()
  net = torchvision.models.resnet18(pretrained=False)

torch.Size([1, 10])


# Model Summary

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = net.to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

train_loader = get_train_loader()
test_loader = get_test_loader()

from tqdm import tqdm
import torch
import torch.nn.functional as F

test_losses = []
test_acc = []

# TODO: remove the issue for import error of this function and use the function
# mentioned inside test.ipynb file. As of now using this function from here.
def test_model(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    pbar = tqdm(test_loader)
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            #pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            #correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    pbar.set_description(desc= f'Loss={loss.item()} , Accuracy={100*correct/total:0.2f}')
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, total,
        100. * correct / total))
    
    test_acc.append(100. * correct / total)
    
# TODO: Take the end epoch value from conf file 
for e in range(1, 70):
    print("epoch no: " , e)
    train_model(model, device, train_loader, optimizer, criterion, e)
    test_model(model, device, test_loader, criterion)
    print("===========================================================\n")



Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/782 [00:00<?, ?it/s]

epoch no:  1


Loss=0.623447060585022 Batch_id=781 Accuracy=93.19: 100%|██████████| 782/782 [00:22<00:00, 37.62it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.0100, Accuracy: 8283/10000 (82.83%)


epoch no:  2


...


Test set: Average loss: 0.0126, Accuracy: 8438/10000 (84.38%)


epoch no:  68


Loss=0.021074414253234863 Batch_id=781 Accuracy=97.37: 100%|██████████| 782/782 [00:22<00:00, 35.19it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.0122, Accuracy: 8403/10000 (84.03%)


epoch no:  69


Loss=0.0038367509841918945 Batch_id=781 Accuracy=97.43: 100%|██████████| 782/782 [00:23<00:00, 33.14it/s]
Loss=0.6852177381515503 , Accuracy=84.11:   0%|          | 0/157 [00:02<?, ?it/s]


Test set: Average loss: 0.0124, Accuracy: 8411/10000 (84.11%)


